In [ ]:
import cv2
import numpy as np
from PIL import Image
from IPython.display import display, Image
import dolphin_gguf.tools.utils as utils

def draw_bbox_on_image(image_path, coordinates, color=(0, 255, 0), thickness=2):
    """
    输入图片路径和边界框坐标，在图片上绘制标记框，并返回标记后的图片（BGR格式）。

    参数:
    image_path (str): 图片文件的完整路径。
    coordinates (tuple): 边界框的坐标 (xmin, ymin, xmax, ymax)。
                         注意: 坐标应该是像素值。
    color (tuple): 边界框的颜色，格式为 (B, G, R)。默认为绿色 (0, 255, 0)。
    thickness (int): 边界框的线宽。默认为 2。

    返回:
    numpy.ndarray: 绘制了边界框的图片数据 (BGR 格式)。
    """
    try:
        # 1. 读取图片
        # cv2.imread 默认读取为 BGR 格式
        image = cv2.imread(image_path)

        if image is None:
            raise FileNotFoundError(f"无法读取图片: {image_path}")

        # 2. 解析坐标
        xmin, ymin, xmax, ymax = coordinates

        # 3. 绘制矩形框
        # cv2.rectangle(img, pt1, pt2, color, thickness)
        # pt1 是左上角 (xmin, ymin)
        # pt2 是右下角 (xmax, ymax)
        marked_image = cv2.rectangle(
            img=image,
            pt1=(xmin, ymin),
            pt2=(xmax, ymax),
            color=color,
            thickness=thickness
        )
        
        return marked_image

    except Exception as e:
        print(f"处理图片时发生错误: {e}")
        return None

def display_cv2_image_in_jupyter(image_data, format='png'):
    """
    将 OpenCV (BGR) 格式的图片在 Jupyter Notebook 中显示。
    """
    if image_data is None:
        return

    # 1. BGR 转换为 RGB（在 Jupyter/浏览器中显示时颜色才正确）
    rgb_image = cv2.cvtColor(image_data, cv2.COLOR_BGR2RGB)

    # 2. 将 NumPy 数组编码为图片格式 (如 PNG)
    # imencode 返回 (成功标志, 编码后的字节流)
    is_success, buffer = cv2.imencode(f'.{format}', rgb_image)
    
    if is_success:
        # 3. 使用 IPython.display.Image 显示
        display(Image(data=buffer.tobytes()))
    else:
        print("图片编码失败，无法显示。")

# 假设您已经有了一张图片并知道其路径
# --- 替换为您的图片路径 ---
# 为了测试，这里创建一个虚拟图片文件路径
test_image_path = '/Users/larry/github.com/dolphin_gguf/demo/demo_1_1600.jpg' 

example_coordinates = (113,591,638,927)#(112,159,638,810)
image = cv2.imread(test_image_path)
height = image.shape[0]
width = image.shape[1]

# bbox = utils.map_bbox_to_original(example_coordinates, width, height)
# example_coordinates = bbox

# 2. 将浮点数列表转换为整数元组
# 推荐使用 int() 向下取整，或者使用 round() 四舍五入，但 int() 更常见于坐标处理
example_coordinates = tuple(int(c* 1.545)  for c in example_coordinates)

# 调用函数获取标记后的图片数据 (BGR 格式)
marked_image_data = draw_bbox_on_image(
    image_path=test_image_path, 
    coordinates=example_coordinates, 
    color=(0, 0, 255),  # 红色 (B=0, G=0, R=255)
    thickness=2
)

# ----------------------------------------------------
# 步骤 3: 在 Jupyter 中显示结果
# ----------------------------------------------------
print("--- 原始图片 ---")
# display_cv2_image_in_jupyter(cv2.imread(test_image_path)) # 显示原始图片

print("\n--- 标记后的图片 ---")
display_cv2_image_in_jupyter(marked_image_data)

# ----------------------------------------------------
# 步骤 4: (可选) 保存标记后的图片
# ----------------------------------------------------
# if marked_image_data is not None:
#     cv2.imwrite('marked_result.png', marked_image_data)
#     print("\n标记后的图片已保存为: marked_result.png")

In [ ]:
#resize image

from PIL import Image
import os

def resize_image(input_path: str, output_path: str, new_max_dim: int, resample_method=Image.LANCZOS):
    """
    将图像缩放到指定的最大维度，并保存为新的低分辨率文件。

    Args:
        input_path (str): 输入图像文件的路径。
        output_path (str): 输出图像文件的路径。
        new_max_dim (int): 缩放后图像的最长边期望的像素数 (例如 896)。
        resample_method: 用于重采样的算法 (默认为 LANCZOS，提供最佳质量)。
    """
    try:
        # 1. 打开图像
        img = Image.open(input_path)
        
        # 确保图像为 RGB 格式，以简化后续处理
        img = img.convert("RGB") 
        
        original_width, original_height = img.size
        print(f"原图尺寸: {original_width} x {original_height}")

        # 2. 计算新的尺寸 (保持宽高比)
        # 找出当前的宽高比
        aspect_ratio = original_width / original_height
        
        if original_width > original_height:
            # 如果宽度更长，则宽度等于 new_max_dim
            new_width = new_max_dim
            new_height = int(new_max_dim / aspect_ratio)
        else:
            # 如果高度更长，则高度等于 new_max_dim
            new_height = new_max_dim
            new_width = int(new_max_dim * aspect_ratio)

        new_size = (new_width, new_height)
        
        # 3. 执行缩放（重采样）
        # 使用 resample_method (如 LANCZOS) 确保缩放质量
        resized_img = img.resize(new_size, resample=resample_method)
        
        print(f"新图尺寸: {resized_img.width} x {resized_img.height}")

        # 4. 保存为新文件
        # 保存时可以指定质量 (quality)，进一步降低文件大小
        resized_img.save(output_path, quality=85) # quality=85 是一个常用的平衡点
        
        print(f"成功保存低分辨率图像到: {output_path}")

    except FileNotFoundError:
        print(f"错误: 找不到文件 -> {input_path}")
    except Exception as e:
        print(f"处理图像时发生错误: {e}")

# --- 示例使用 ---
if __name__ == "__main__":
    TARGET_MAX_DIM = 1600

    # 原始: 1654 × 2369像素
    INPUT_FILE = "/Users/larry/github.com/dolphin_gguf/demo/demo_1.png" 
    
    OUTPUT_FILE = f"/Users/larry/github.com/dolphin_gguf/demo/demo_1_{TARGET_MAX_DIM}.jpg" 
    
    resize_image(INPUT_FILE, OUTPUT_FILE, TARGET_MAX_DIM)
    